<a href="https://colab.research.google.com/github/bwsi-hadr/01-Intro-to-python/blob/master/01b_Intro_to_pandas_with_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Pandas

Pandas is a package built on top of numpy which incorporates many additional features which make it more convenient for data analysis. One main addition is the inclusion of row and column labels, which make pandas function much like spreadsheet software. It also includes tools for much more complex computations than regular numpy allows

In [ ]:
import pandas as pd
import numpy as np
# by default, pandas only shows the beginning and end of a large dataframe
# here we set it to None so that we see all of the dataframe
# change this to an integer to limit the number of elements it displays
pd.options.display.max_rows = None 

## Importing data
There are a lot of built-in functions for importing data to pandas, for example, from files, database, and web sources. For a full list of the supported methods, see [the documentation on pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).

In this example, we're going to load data formatted as a csv from a website URL. In particular, it is a direct link to a file hosted in our course github repository.

The data we're loading is a simplified pokedex which includes the number, name, base stats, types, abilities, and colors of each pokemon. We'll be using this data to explore the functionality of pandas. 

The primary class of pandas is the DataFrame, which functions like a spreadsheet (like in excel). It allows for named rows and columns. Data types between two different columns can vary, but entries within the same column should have the same type. 

In [ ]:
# URL for data
pokedex_url = 'https://raw.githubusercontent.com/bwsi-hadr/01-Intro-to-python/master/pokedex.csv?token=AK6STNOT3VACKCD37B6THDK5DYG4M' 
# instead of a url, you could instead put a path to a csv file on the local system where python is running. However, since colab is on the cloud, it's easier to use a url
pokedex_df = pd.read_csv(pokedex_url, index_col='Index') # create a dataframe from a csv, set the column titled "Index" as the row labels

pokedex_df = pokedex_df.fillna('') #fill in the NaN values with empty string 
pokedex_df # prints out the dataframe

## Indexing
Like numpy, you can index a dataframe to select elements or subsets of entries. There are a number of ways to index pandas dataframes. The two following methods are recommended:
`.loc[]` and `.iloc[]`

### .loc[ ] name-based indexing
The first way, using `.loc[a,b]` indexes based on the __values__ of the row index and column names. This allows us to get entries where the row index value equals `a` and the column name equals `b`.

In [ ]:
# Get the entry for index value 745.1 - Lycanroc (midnight)
pokedex_df.loc[745.1]

In [ ]:
# You can get specific column values for a single entry
pokedex_df.loc[250,['Pokemon','Type I','Type II']]

In [ ]:
# you can use the : notation to slice multiple rows and columns
pokedex_df.loc[690:700,'Pokemon':'Total']

In [ ]:
# Similarly to numpy, you can use the : notation without indices on a side to mean "from beginning" or "to end"
pokedex_df.loc[:10,"Ability I":]

In [ ]:
# and : by itself gives all of the entries in that dimension
pokedex_df.loc[:,'Pokemon'].head(10) #just return top 10 entries

### .iloc[ ] integer indexing

The second way, using `.iloc[i,j]` uses the numpy-style integer indices to get a value. In this case, iloc takes integer values from 0 to the number of entries in that dimension. 

Like numpy, `.iloc[i,j]` gives the entry at the $i^{th}$ row and $j^{th}$ column.

In [ ]:
pokedex_df.iloc[100] # gets 101st row (recall python counts from 0)

In [ ]:
# The : style indexing also works for iloc
pokedex_df.iloc[25:30, 0:8]

## Adding columns, rows, and changing values
You can add another column like:
```
dataframe['NewColumnName'] = value
```
`value` can be a single value, an array of the same size as the index of `dataframe`, a pandas `Series`, or a dictionary mapping `index: value` for entries in the dataframe index

In [ ]:
pokedex_df['Nickname'] = None
pokedex_df

In [ ]:
# you can change the value of an entry using indexing and assignment =
pokedex_df.loc[4,'Nickname'] = 'Abby' # indexing using .loc[] pokedex entry 4, 'Nickname column'
pokedex_df.iloc[113,-1] = 'Terry' # indexing using .iloc[] 113rd row, last column
pokedex_df.iloc[[3,113],:] 

In [ ]:
# you can add another dataframe on to the end using append()
new_pokemon = pd.DataFrame([{'Pokemon':'Grookey', 'Type I':'Grass', 'Color':'Green'},
                            {'Pokemon':'Scorbunny', 'Type I':'Fire', 'Color':'Red'},
                            {'Pokemon':'Sobble', 'Type I':'Water', 'Color':'Blue'}], 
                           index=[803.0, 804.0, 805.0]) # create a dataframe with new entries using a list of dictionaries
pokedex_df = pokedex_df.append(new_pokemon, sort=False) # the sort=False keyword prevents the columns from being reordered alphabetically
pokedex_df = pokedex_df.fillna('').drop_duplicates() # some cleanup operations, replace NaN with emptry strings, and drop duplicate rows
pokedex_df.tail()

## Changing index column
You can change the index column of the dataframe with `.set_index()` or reset it to default numerical indexing with `.reset_index()`.

Note however, that these operations create a __copy__ of the original dataframe, and do not change the dataframe "in place" (i.e. doesn't change the original object)

In [ ]:
# reset index to be default numerical index
reindexed_pokedex = pokedex_df.reset_index()
reindexed_pokedex.head()

In [ ]:
# Change the index to the Pokemon name column
pokedex_indexed_by_name = pokedex_df.set_index('Pokemon')
pokedex_indexed_by_name

In [ ]:
# note that the original dataframe was not affected
pokedex_df

In [ ]:
# drop last 3 rows
pokedex_df = pokedex_df.drop([803, 804, 805])
pokedex_df.tail()

## Logical indexing
You can test conditionals on a column, for example, testing if the value of the column is greater than a value. What's returned is a boolean column of the same size, with a value of `True` wherever it is true, and `False` otherwise. (This can similarly be done in numpy).



In [ ]:
pokedex_df['Total'] > 520

If you take that boolean column and use it to index the dataframe, you can get all of the entries that satisfy that condition.

In [ ]:
pokedex_df.loc[pokedex_df['Total']>=520] # all of the pokemon with total base stats greater than 500

In [ ]:
# Get all of the pokemon of a given color
pokedex_df.loc[pokedex_df['Color']=='Black']

### Compound boolean queries
You can use boolean operations (OR, AND, NOT) to combine conditions

We will look for pokemon with specific types to demonstrate compound queries

### OR queries
In the following example, we query for all pokemon who have at least part "Dragon" typing.

The vertical line | represents the element-wise "OR" operation. "OR" operations require either or both statements on either side of the operation to be true.

Thus, the below query 

```query = (pokedex_df['Type I']=='Dragon') | (pokedex_df['Type II']=='Dragon')```

asks, for each row, if 'Type I' is equal to "Dragon" OR 'Type II' is equal to "Dragon"

In [ ]:
query = (pokedex_df['Type I']=='Dragon') | (pokedex_df['Type II']=='Dragon')
pokedex_df.loc[query] # Get all of the entries corresponding to the above query

In [ ]:
(pokedex_df['Type I']=='Dragon').all()

### AND queries
The ampersand `&` character represents the element-wise boolean operation "AND". In order for an "AND" statement to be true, both conditions on either side of the operation must be true.

We can combine boolean operations to construct complex queries: for example, getting all pokemon who are both Dragon and Flying types. The query below looks for pokemon who are EITHER (Primary Type "Flying" AND Secondary Type "Dragon") OR (Primary Type "Dragon" AND Secondary Type "Flying")

Note that parentheses should be used to group operations so that the order of operations is clear

In [ ]:
# you can use the backslash operator \ to continue code on the next line
query = ((pokedex_df['Type I']=='Flying') & (pokedex_df['Type II']=='Dragon'))\
        | ((pokedex_df['Type I']=='Dragon') & (pokedex_df['Type II']=='Flying'))
pokedex_df.loc[query]

### NOT queries
There are two ways to do negation. Consider the following query for selecting all pokemon who are Fire type, but not Flying

```
query = ((pokedex_df['Type I']=='Fire') & ~(pokedex_df['Type II']=='Flying'))\
        | ((pokedex_df['Type I']!='Flying') & (pokedex_df['Type II']=='Fire'))
```

The first way to do negation is with the tilde ~ symbol, which negates the subsequent boolean statement:
We use this in `~(pokedex_df['Type II']=='Flying')`.
The parentheses determine order of operations. First we find all entries where their secondary type is "Flying"
Then the tilde in front negates that statement, flipping all True values to False, and vice versa

The second way to do negation is using the != notation, which represents "not equal to"
We use this in `(pokedex_df['Type I']!='Flying')`
This finds all entries where their primary type is not equal to "Flying"

In [ ]:
query = ((pokedex_df['Type I']=='Fire') & ~(pokedex_df['Type II']=='Flying'))\
        | ((pokedex_df['Type I']!='Flying') & (pokedex_df['Type II']=='Fire'))

pokedex_df.loc[query]


## Sorting
Dataframes can be sorted by value

In [ ]:
pokedex_df.sort_values('Pokemon', ascending=False) # Sort by name descending

In [ ]:
# You can sort by multiple columns
pokedex_df.sort_values(['Type I','Type II', 'Total'], ascending=False) # Sort by type combination, then stats descending

## Calculations and operations
There are various ways to do math to the dataframes

### Column operations
You can perform basic math element-wise between columns, such as addition, subtraction, multiplication, and division

In [ ]:
pokedex_df['Total']/ pokedex_df['HP'] # computes the ratio between total stats and HP

### Universal functions
Universal functions can operate on entire dataframes or slices of dataframes. They return a dataframe copy with the same index as the original dataframe. Numpy universal functions can be applied to any numeric dataframe or slice.

In [ ]:
np.log(10)

In [ ]:
np.log(pokedex_df.loc[:,'HP':'Total'].astype(float), axis=0) # take natural log of all stats
# again, these functions operate on each element independently, and return a dataframe with the same index and size of input

### .apply()
You can apply an arbitrary function to each row or column using the `.apply()` function.

If you pass the name of a function (e.g. np.mean, or the name of any function you've written), you can apply it to either each column, or each row using `.apply()`

In [ ]:
pokedex_df.loc[:,'Atk':'Speed'].apply(np.mean) # applies the function np.mean() to each column

In [ ]:
pokedex_df.loc[:,'Atk':'Speed'].apply(np.mean, axis=1) # if you wanted to apply it to each row instead, use the argument axis=1

The above example is actually a bit unnecessary. There are several built-in functions in dataframes, including a `.mean()` which has the same functionality.

In [ ]:
pokedex_df.loc[:,'Atk':'Speed'].mean()

In [ ]:
pokedex_df.loc[:,'Atk':'Speed'].mean(axis=1) #applies per row

Most [descriptive statistics](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html#computations-descriptive-stats) (median, mode, standard deviation, count, quantile, etc) have built-in functions. Where `.apply()` really shines, however, is for applying custom functions. 

To show that functionality, let's take a sidebar and talk about lambda functions.

#### Lambda functions
Sometimes you want to define a function, but you don't want to write a whole 
```
def functionname(inputs):
  ...
  return outputs
```

In those situations, you can write lambda (anonymous) functions. The syntax for lambda functions is:

```
functionname = lambda x: operations_on_x
```

For example, if you wanted to write a function that squares the input, you could write:

In [ ]:
square_it = lambda x: x**2 # recall that ** is exponentiation in python, this function returns the square of the input
for num in range(10):
  print(square_it(num))

This is especially useful if some function takes another function as an argument, like panda's `.apply()`

The typical usage of `.apply()` is:

`dataframe_obj.apply(functionname)`, which calls `functionname` on each column of `dataframe_obj`

However, you can replace `functionname` with a lambda function. In this example, we use the following lambda function:

```lambda col: [np.min(col), np.max(col)]```

The word `col` represents the argument, and then to the right of the colon, we return a list with two elements: the minimum and maximum values of the column


In [ ]:
minmax_stats = pokedex_df.loc[:,'HP':'Total'].apply(lambda col: [np.min(col), np.max(col)])
print(minmax_stats)

the above can be expanded into a dataframe:

In [ ]:
pokedex_df.loc[:,'HP':'Total'].apply(lambda col: [np.min(col), np.mean(col), np.max(col)], result_type='expand')

### Groupby
Dataframes can be grouped by column values, and operations can be applied to each group

In [ ]:
pokedex_df.groupby(['Type I','Type II'])['Total'].count()

In [ ]:
pokedex_df.groupby(['Type I','Type II'])['Total'].apply(lambda col: [np.min(col), np.max(col)])

# Intermediate Pandas
In this section, we'll cover some more advanced topics in pandas, including 
*   Multi-indexing
*   Joining and merging
*   Pivot tables

We'll be introducing some more data to play with, based on state populations over time


In [ ]:
state_populations_url = 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv'
state_abbrevs_url = 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv'
state_areas_url = 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv'

state_areas = pd.read_csv(state_areas_url, index_col='state')
state_areas.head() # just print top 5 rows

In [ ]:
state_abbrevs = pd.read_csv(state_abbrevs_url)
state_abbrevs.head()

In [ ]:
state_populations = pd.read_csv(state_populations_url) 
state_populations.head(10) #print first 10 rows

In [ ]:
state_populations.tail()

Note that the state populations are just listed by abbreviations. It may be useful to include the state names as a column in a table. We have this information in the `state_abbrevs` table. We'd like to merge it with the population data.

The function `pd.merge()` merges two dataframes using set logic. 

In [ ]:
merged = pd.merge(state_populations, state_abbrevs, how='outer',
                 left_on='state/region', right_on='abbreviation')
# The arguments left_on and right_on specify what columns to join on. It looks for matching values in each dataframe
# and fills in the additional columns of the merged dataframe based on the matching data
# The 'how' argument specifies what kind of join. Outer means keep all entries even if one does not show up in the other
# conversely, 'inner' discards rows where a corresponding entry does not exist in the other
merged

In [ ]:
# we can also drop the unnecessary extra abbreviation column now
merged = merged.drop('abbreviation', axis=1) #drop from columns
merged.head()

In [ ]:
# we can check if there were any missing values
merged.isnull().any()

In [ ]:
# let's see where the issue is. What entries are missing population entries?
merged.loc[merged['population'].isnull()]

In [ ]:
# what about where there's no state entry?
merged.loc[merged['state'].isnull(), 'state/region']

In [ ]:
# more concicely, we can use .unique() to only see the unique values
merged.loc[merged['state'].isnull(), 'state/region'].unique()

In [ ]:
# We can fix this easily
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

In [ ]:
# now let's combine the data with the area data
final = pd.merge(merged, state_areas, on='state', how='left')
final.head()

In [ ]:
# we can use the .query() syntax to search for specific data
data2010 = final.query("year == 2010 & ages == 'total'") # specify column name 
data2010.set_index('state', inplace=True) # equivalent to data2010 = data2010.set_index('state')
data2010.head()

In [ ]:
# Let's compute the density of each state and then sort by density
density = data2010['population'] / data2010['area (sq. mi)']
density.sort_values(ascending=False, inplace=True)
density

In [ ]:
density.tail()

### Exercises
Compute the total population of the entire united states, as well as its population density for each year.

Sort the states by population density.

What year saw the biggest change in population for the US as a whole?

Which states had the biggest percentage change in population in that year?

In [ ]:
state_areas.sum().values

In [ ]:
# total population
final.loc[(final['state/region']!='USA')&(final['ages']=='total')]\
     .groupby('year')['population'].sum()

In [ ]:
final.query("state!='United States' & ages=='total'").groupby('year')['population'].sum()

In [ ]:
final['density'] = final['population']/final['area (sq. mi)']
final.loc[(final['state']=='United States') & (final['ages']=='total')]

In [ ]:
final.loc[final['state']=='United States','area (sq. mi)'] = state_areas.sum().values

In [ ]:
final.loc[(final['state']=='United States') & (final['ages']=='total')]\
          .sort_values('year')['population'].diff()

In [ ]:
idx_max = final.loc[(final['state']=='United States') & (final['ages']=='total')]\
          .sort_values('year')['population'].diff().idxmax()


In [ ]:
year_max = final.loc[idx_max,'year']
year_max

In [ ]:
data_year_max = final.loc[(final['year']==year_max)&(final['ages']=='total')]
data_year_before = final.loc[(final['year']==year_max-1)&(final['ages']=='total')]

In [ ]:
data_year_max = data_year_max.copy() # make a copy so we can change things destructively
data_year_max.loc[:,'percent_change'] = 100*(data_year_max['population'].values  - data_year_before['population'].values)/data_year_before['population'].values


In [ ]:
data_year_max

In [ ]:
data_year_max.loc[data_year_max['percent_change'].abs().idxmax()]

## Multiindexing
You can use multiple columns to define an index, as long as each row has a unique combination of values within those columns. In the following example, we will use the columns "state/region", "year", and "ages" of the population dataframe as an index

We use the `.set_index(['column_name1', 'column_name2, ...])'` command to create a multiindex from existing columns.

Note that many operations require a multiindex to be sorted, so we also call `.sort_index()` on the dataframe. Also, `.set_index()` and `.sort_index()` return a _copy_ of the dataframe, so we assign the output to itself to update the value.

In [ ]:
# set index to use multiple columns
final = final.set_index(['state/region','year','ages']) # overwrite old version with new version with index set
final = final.sort_index() # replace itself with a version with a sorted index
final.tail(10) #print last 10 entries

In [ ]:
final.loc[('MA',2009,'total'):('WY',2011,'total')]

### Stacking and unstacking
Columns can also have multiple levels. In fact, the columns can be thought of as just another index that runs horizontally. This is most easily illustrated by stacking, unstacking, and transposing.

You can convert a level of the row index to a column level using the `.unstack()` command. Conversely, `.stack()` moves a column level to the index


In [ ]:
unstacked = final.unstack(level='ages') # you can unstack by level name; this moves the 'ages' categories into the column index
unstacked

In [ ]:
restacked = unstacked.stack('ages', dropna=False) # stacks the column level back into the row index
# by default, .stack() omits entries with NaN. If you want to preserve the NaN entries, use the option dropna=False
restacked == final # just like the original state_populations dataframe

In [ ]:
final.unstack(level=0) # you can also unstack using integer levels. Here we unstacked level 0, corresponding to state/region

You can switch the row and column index using the transpose function `.T`

In [ ]:
final.unstack(level=0).T

### Pivoting
Pivoting is another advanced way to transform data. It creates a new dataframe using the values of columns as `index` and `column` values. 

You can then aggregate using `aggfunc` the elements of another column, which becomes the `values` of this new dataframe.


In [ ]:
# let's create a table which lists all pokemon for given type combinations
pokedex_df.pivot_table(values='Pokemon', # line breaks within parentheses automatically continue onto next line
                       index='Type I', # values of Type I become index entries
                       columns='Type II', # likewise Type II become columns
                       aggfunc=lambda x:", ".join(x)).fillna('') # the lambda function here just joins a list of strings using a comma
                      # we also fill nan values with empty string for aesthetics

In [ ]:
# compute the average stats for each type combination
pokedex_df.unstack()
pokedex_df.pivot_table(values='Total',
                       index='Type I', 
                       columns='Type II', 
                       aggfunc=lambda x: np.mean(x)).fillna('')

In [ ]:
# You can compute multiple aggregations on multiple columns by passing a dict
# into the aggfunc argument: aggfunc={'column_name':[list_of_function_names]}
# Note that in this case, the 'value' argument is ignored
pokedex_df.pivot_table(index='Type I', 
                       columns='Type II', 
                       aggfunc={'Total':[lambda x: np.mean(x),lambda x: np.std(x)], 
                                'Atk':[lambda x: np.mean(x),lambda x: np.std(x)]}).fillna('')

# Exercise
Using this data (or any other data you're interested in and you find online), put together an analysis using pandas that investigates an interesting question.

It can be serious or silly. Like "Do states with higher population density tend to vote democratic?" (you'd need to pull additional data for this) or "Do pokemon whose names start with the letter D have better stats than those who start with F?"

We'll ask some volunteers to present their analysis at the end
